In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Gross
import sys
sys.path.append("..")

In [4]:
import random
from world_rando import generate
from rom_tools.rom_manager import RomManager, region_map_locs
from rom_tools import rom_data_structures, leveldata_utils
from world_rando.room_dtypes import convert_rooms
from rom_tools.address import Address
from encoding import free_space
from world_rando import map_viz
from typing import cast

In [5]:
from rom_tools import rom_edit

In [6]:
rom_m = RomManager("../../roms/Super Metroid (JU) [!] (patched - Subversion 1.2).smc", "../../roms/sm_test_edit.smc")
# rom_m = RomManager("../../roms/sm_clean.smc", "../../roms/sm_test_edit.smc")

looks like subversion


In [8]:
print("Parsing original rooms")
parsed_rooms = rom_m.parse()

Parsing original rooms
Parsing: save_station_0x44c5
Using pointer_parser at 0x44c5
Parsing: room_header_0x79330
Using parse_int at 0x79330
Got: 27, Size: 1 from parse_int at 0x79330

Using enum_parser at 0x79331
Got: 1, Size: 1 from enum_parser at 0x79331

Using parse_int at 0x79332
Got: 17, Size: 1 from parse_int at 0x79332

Using parse_int at 0x79333
Got: 14, Size: 1 from parse_int at 0x79333

Using parse_int at 0x79334
Got: 1, Size: 1 from parse_int at 0x79334

Using parse_int at 0x79335
Got: 1, Size: 1 from parse_int at 0x79335

Using parse_int at 0x79336
Got: 112, Size: 1 from parse_int at 0x79336

Using parse_int at 0x79337
Got: 160, Size: 1 from parse_int at 0x79337

Using enum_parser at 0x79338
Got: set(), Size: 1 from enum_parser at 0x79338

Using pointer_parser at 0x79339
Parsing: door_list_0x7b420
Using list_parser at 0x7b420
Parsing: door_0x1c96c
Using pointer_parser at 0x1c96c
Parsing: room_header_0x79bb8
Using parse_int at 0x79bb8
Got: 84, Size: 1 from parse_int at 0x79bb

AssertionError: failed to parse 0x7fee8

In [85]:
rooms: list[rom_data_structures.RoomHeader] = []
for key in parsed_rooms.keys():
    if key.startswith("room_header"):
        # print(key)
        rooms.append(cast(rom_data_structures.RoomHeader, parsed_rooms[key]))

In [86]:
for room_header in rooms:
    state_chooser = cast(rom_data_structures.StateChooser, room_header.state_chooser)
    default = cast(rom_data_structures.RoomState, state_chooser.default)
    level_data = default.level_data
    # print(type(level_data))
    level_array = cast(leveldata_utils.LevelArrays, level_data.level_array)
    # for row in level_array.bts:
    #     print(row)
    print(f"{room_header.name}: {len(level_array.bts)} {len(level_array.bts[0])}")

room_header_0x791f8: 144 80
room_header_0x792fd: 80 80
room_header_0x7990d: 96 48
room_header_0x799bd: 16 112
room_header_0x79969: 64 16
room_header_0x79938: 16 16
room_header_0x79ad9: 64 192
room_header_0x79b9d: 48 16
room_header_0x79c35: 16 16
room_header_0x79fe5: 16 16
room_header_0x7a011: 80 32
room_header_0x7a051: 16 16
room_header_0x7a22a: 16 16
room_header_0x79c5e: 48 32
room_header_0x79c89: 16 16
room_header_0x79bc8: 48 32
room_header_0x79c07: 32 16
room_header_0x79cb3: 112 112
room_header_0x79d19: 80 160
room_header_0x79d9c: 64 16
room_header_0x79dc7: 16 48
room_header_0x79b5b: 32 144
room_header_0x7a0a4: 48 16
room_header_0x79e11: 32 32
room_header_0x79e52: 128 64
room_header_0x79e9f: 128 48
room_header_0x79f11: 16 32
room_header_0x79f64: 48 48
room_header_0x7a1ad: 32 16
room_header_0x7a1d8: 16 16
room_header_0x7a107: 16 16
room_header_0x797b5: 16 16
room_header_0x7975c: 48 32
room_header_0x796ba: 48 144
room_header_0x799f9: 64 128
room_header_0x7dede: 48 96
room_header_0x7de

In [87]:
room_header = rooms[5]
level_array = cast(leveldata_utils.LevelArrays, room_header.state_chooser.default.level_data.level_array)
for row_i in range(16):
    for col in level_array.layer1:
        tile = col[row_i]
        print(tile.tile_type or " ", end="  ")
    print()

8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  
8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  
8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  
8  8  8                                8  8  8  
8  8  8                                8  8  8  
8  8  8                                8  8  8  
8                                         12  9  
8                                         13  9  
8                                         13  9  
8                                         13  9  
8  8  8  8  8  8  8  9  9  8  8  8  8  8  8  8  
8  8  8  8  8  8  8        8  8  8  8  8  8  8  
8  8  8  8  8  8  8        8  8  8  8  8  8  8  
8  8  8  8  8  8  8        8  8  8  8  8  8  8  
8  8  8  8  8  8  8        8  8  8  8  8  8  8  
8  8  8  8  8  8  8  9  9  8  8  8  8  8  8  8  
